In [1]:
import torch
import torchvision
import numpy as np
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms
import sys
from tqdm import tqdm
import numpy as np
sys.path.append("/workspaces/ood/")

%load_ext autoreload
%autoreload 2
device = torch.device("cuda")
torch.cuda.is_available()

True

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)

In [3]:
cifar_data_train = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform
)

In [4]:
wide_resnet50 = torchvision.models.wide_resnet50_2(pretrained=False).to(device)
wide_resnet50.load_state_dict(torch.load('/workspaces/ood/data/models/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth'))

for name, param in wide_resnet50.named_parameters():
    param.requires_grad = False

In [9]:
from fastflow.extract_features import create_feature_dataset

model_name = 'wide_resnet50'
dataset_name = 'cifar10train'
layers = ['layer2', 'layer3', 'layer4']
num_images_per_class=1000
out_name = '../data/feature_maps/' + '_'.join([dataset_name] + [model_name] + layers) + f'_{num_images_per_class}_img_per_class.npz'

create_feature_dataset(model=wide_resnet50, layers=layers, dataset=cifar_data_train, num_images_per_class=num_images_per_class, out_name=out_name, device=device)

  0%|          | 0/50000 [00:00<?, ?it/s]

saving to ../data/feature_maps/cifar10train_wide_resnet50_layer2_layer3_layer4_1000_img_per_class.npz...


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 50000/50000 [02:23<00:00, 348.08it/s] 


RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 8036188160 bytes. Error code 12 (Cannot allocate memory)

In [6]:
test = np.load('/workspaces/ood/data/feature_maps/cifar10train_wide_resnet50_layer2_layer3_layer4.npz')

In [8]:
test['layer4'].dtype

dtype('float32')

In [8]:
out_name

'../data/feature_maps/cifar10train_wide_resnet50_layer2_layer3_layer4_1000_img_per_class.npz'